In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.linalg import norm
import itertools
from multiprocessing import Pool, freeze_support
from tqdm import tqdm
import os
from IPython import display
import seaborn as sns
from IPython.display import HTML
from glob import glob
from helper_util import Func_GaussElimination,plotter_animation_contour
import json
from func_list_jax import *
#from func_list_vect import visualizer
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import cg

In [ ]:
v,args,cg_iter,nx,ny,dx,dy,np_,nu,X,Y,ip,iu,iv,idu,dt,p,u,qi,bcL,bcD,uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B=data_unloader(**json.load(open('/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/data.json')))

Adv_O = LinearOperator((nu,nu), matvec=lambda x: Adv_Vec(x, *args))
Grad_O = LinearOperator((nu,np_), matvec=lambda x: Grad_Vec(x, *args))
Div_O = LinearOperator((np_,nu), matvec=lambda x: Div_Vec(x, *args))
Laplace_O = LinearOperator((nu,nu), matvec=lambda x: Laplace_Vec(x, *args))
S_O=S_operator_O(args)
R_O=R_operator_O(args)
R_inv_O=R_inv_operator_O(args)

Div_Grad=Div_O@Grad_O
Div_Rinv_Grad=Div_O@R_inv_O@Grad_O
#B = LinearOperator((nu,nu), matvec=lambda x: 1-(dt/2)*Laplace_Vec(x, *args))
#C = LinearOperator((nu,nu), matvec=lambda x: 1+(dt/2)*Laplace_Vec(x, *args))

In [ ]:
A_n=Adv_O.matvec(qi)
Lu_n=Laplace_O.matvec(qi)+v*bcL
uf=(qi + dt*(A_n+Lu_n))[:,0]
D_rhs=((Div_O.matvec(uf)+bcD)/dt)[0]
P, exit_code = cg(Div_Grad, D_rhs)
u_new=(uf-dt*(Grad_O.matvec(P))).reshape(nu,1)

A_n_1 = Adv_O.matvec(qi)
A_n= Adv_O.matvec(u_new)

RHS_b = (S_O.matvec(u_new) + dt*(3*(A_n)-(A_n_1))/2 +dt*v*bcL)
uf,exit_code  = cg(R_O, RHS_b,maxiter=cg_iter)

## fractial step: stage 2
RHS_b = (Div_O.matvec(uf)+bcD[:,0])/dt
pnew,_= cg(Div_Rinv_Grad,RHS_b,P,maxiter=cg_iter)


In [ ]:
RHS_b

In [ ]:
drG1=Div_Rinv_Grad.matvec(P)

In [ ]:
drG2=Div_Vec(R_inv_operator(Grad_Vec(P,*args),*args),*args).T

In [ ]:
A_n=Adv_Vec(qi, *args)
Lu_n=Laplace_Vec(qi,*args)+bcL
uf=qi + dt*(A_n+v*Lu_n)
D_rhs= (Div_Vec(uf,*args)+bcD)/dt
P=CG_solver_all([Grad_Vec,Div_Vec],D_rhs,p,args,cg_iter)
u_new=uf-dt*Grad_Vec(P,*args)


A_n_1 = Adv_Vec(qi, *args)
A_n= Adv_Vec(u_new, *args)
RHS_b1 = S_operator(u_new,*args) + dt*(3*(A_n)-(A_n_1))/2 +dt*v*bcL
uf    =CG_solver(R_operator,RHS_b1,u_new,args,cg_iter)


RHS_b2 = (Div_Vec(uf,*args)+bcD)/dt
pnew  = CG_solver_all([Grad_Vec,R_inv_operator,Div_Vec],RHS_b,P,args,cg_iter)

In [ ]:
RHS_b1

In [ ]:
pnew  = CG_solver_all([Grad_Vec,R_inv_operator,Div_Vec],RHS_b,P,args,cg_iter)

In [ ]:
A_n_1 = Adv_Vec(qi, uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B,nu,iu,iv,nx,ny,dx,dy,dt,v)
A_n= Adv_Vec(u_new, uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B,nu,iu,iv,nx,ny,dx,dy,dt,v)
RHS_b = S_operator(u_new,uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B,nu,iu,iv,nx,ny,dx,dy,dt,v) + dt*(3*(A_n)-(A_n_1))/2 +dt*v*bcL
args=nu,iu,iv,nx,ny,dx,dy,dt,v

In [ ]:

## time stepping using fractial step
res=[]
nt=200000
for it in range(2):
    ## fractial step: stage 1
    A_n_1 = Adv(qi, uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B,nu,iu,iv,nx,ny,dx,dy,dt,v)
    A_n= Adv(u_new, uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B,nu,iu,iv,nx,ny,dx,dy,dt,v)
    RHS_b = S_operator(u_new,nu,iu,iv,nx,ny,dx,dy,dt,v) + dt*(3*(A_n)-(A_n_1))/2 +dt*v*bcL
    args=nu,iu,iv,nx,ny,dx,dy,dt,v
    u_int=np.random.rand(u_new.shape[0],u_new.shape[1])
    uf    = CG_solver(R_operator,RHS_b,u_int,args,cg_iter)

    ## fractial step: stage 2
    RHS_b = (Div(uf,np_,nu,nx,ny,dx,dy,iu,iv,ip,dt,v)+bcD)/dt
    
    args1=np_,nu,nx,ny,dx,dy,iu,iv,ip,dt,v
    args2=nu,iu,iv,nx,ny,dx,dy,dt,v
    args3=np_,nu,nx,ny,dx,dy,iu,iv,ip,dt,v
    
    
    pnew  = CG_solver_all([Grad,R_inv_operator,Div],RHS_b,P,args1,args2,args3,cg_iter)

    ## fractial step: stage 3 (assemble u_new)
    qi=u_new
    P=np.random.rand(pnew.shape[0],pnew.shape[1])
    u_new = uf-dt*R_inv_operator(Grad(pnew,np_,nu,nx,ny,dx,dy,iu,iv,ip,dt,v),nu,iu,iv,nx,ny,dx,dy,dt,v)
    if it%1==0:
        norm_l2 = (norm(u_new-qi))/(norm(qi))
        res.append(norm_l2)
        with open('test1.npy', 'wb') as f:
            np.save(f, qi)
            np.save(f, u_new)
#pd.DataFrame({'u_new':u_new[:,0]}).to_csv('latest_data.csv')

In [ ]:
#file='/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/snapshots/Re_40/ulist_105.npy'
file='/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/test_2.npy'
with open(file, 'rb') as f:
    qi= np.load(f,allow_pickle=True)
    u_new = np.load(f,allow_pickle=True)

In [ ]:
u_new.shape

In [ ]:
#s=glob('/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/snapshots/*')
#s=[int(x.split('ulist_')[-1].split('.')[0]) for x in s ]
#max_u=np.max(s)
#file=f"/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/snapshots/ulist_{max_u}.npy"
#with open(file, 'rb') as f:
#    u_new = np.load(f,allow_pickle=True)
file='/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/test_2.npy'
with open(file, 'rb') as f:
    qi= np.load(f,allow_pickle=True)
    u_new = np.load(f,allow_pickle=True)
nx = 128
ny = 128
## Pointer and Grid
ip, iu, iv,idu = GenPointer(nx, ny)

X,Y=np.meshgrid(np.linspace(0,1,nx), np.linspace(0,1,ny), indexing='ij')

u_new_int,v_new_int=inter_velocity(u_new,nx,ny,iu,iv,idu)
u_new_int=u_new_int.reshape(nx,ny)
v_new_int=v_new_int.reshape(nx,ny)
fig,ax=plt.subplots(1,1)
#cp = ax.contour(X, Y, curl_visual,cmap='coolwarm')
cp = ax.contourf(X, Y, u_new_int)
fig.colorbar(cp)
ax.set_title('V-Velocity 30X29 Grid (Re=400,dt=0.0001 )')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
u_new.shape[0]

In [ ]:
visualizer(u_new, *args)

In [ ]:
fig,ax=plt.subplots(1,1)
#cp = ax.contour(X, Y, curl_visual,cmap='coolwarm')
cp1 = ax.streamplot(X.T, Y.T, u_new_int.T, v_new_int.T, color=np.sqrt(u_new_int**2+v_new_int**2),density=1.2)
cp = ax.contourf(X, Y, np.sqrt(u_new_int**2+v_new_int**2),cmap='coolwarm')
fig.colorbar(cp)
ax.set_title('Streamlines with color showing velocity magnitude')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
plt.streamplot(X.T, Y.T, u_new_int.T, v_new_int.T, color=np.sqrt(u_new_int**2+v_new_int**2),density=1.2)
fig.colorbar(cp)
plt.xlim((0, 1))
plt.ylim((0, 1))
plt.show()

In [ ]:
#s=glob('/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/snapshots/*')
#s=[int(x.split('ulist_')[-1].split('.')[0]) for x in s ]
#max_u=np.max(s)
#file=f"/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/snapshots/ulist_{max_u}.npy"
#with open(file, 'rb') as f:
#    u_new = np.load(f,allow_pickle=True)

nx = 30
ny = 29
## Pointer and Grid
ip, iu, iv,idu = GenPointer(nx, ny)

X,Y=np.meshgrid(np.linspace(0,1,nx), np.linspace(0,1,ny), indexing='ij')
u_vec=np.reshape(u_new[0:iu[-1,-1]+1], (nx-1,ny), order='C')
new_row=np.zeros((ny))
u_vec=np.vstack([u_vec, new_row])

v_vec=np.reshape(u_new[iu[-1,-1]+1:], (nx,ny-1), order='C')
new_row=np.zeros((nx,1))
v_vec=np.hstack([v_vec, new_row])

fig,ax=plt.subplots(1,1)
#cp = ax.contour(X, Y, curl_visual,cmap='coolwarm')
cp = ax.contourf(X, Y, v_vec)
fig.colorbar(cp)
ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
fig,ax=plt.subplots(1,1)
plt.streamplot(X.T, Y.T, u_vec.T, v_vec.T,density=1)
plt.xlim((0, 1))
plt.ylim((0, 1))
ax.set_title('Streamlines')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')

plt.show()

In [ ]:
u_new_s=np.arange(0,len(u_new)).reshape(len(u_new),1)
u_vec_s=np.reshape(u_new_s[0:iu[-1,-1]+1], (nx-1,ny), order='C')
u_vec_s=np.vstack([np.zeros((ny)),u_vec_s])

In [ ]:
#v_new_s=np.arange(0,len(u_new)).reshape(len(u_new),1)

v_new_s=np.reshape(u_new_s[iu[-1,-1]+1:], (nx,ny-1), order='C')
v_new_s=np.hstack([np.zeros((nx,1)),v_new_s])


In [ ]:
fig,ax=plt.subplots(1,1)
#cp = ax.contour(X, Y, curl_visual,cmap='coolwarm')
cp = ax.contourf(X,Y,u_vec_s)
fig.colorbar(cp)
ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()
s=2

In [ ]:
u_new_int,v_new_int=inter_velocity(u_new,nx,ny,iu,iv,idu)
u_new_int=u_new_int.reshape(nx,ny)
v_new_int=v_new_int.reshape(nx,ny)

In [ ]:
fig,ax=plt.subplots(1,1)
#cp = ax.contour(X, Y, curl_visual,cmap='coolwarm')
cp = ax.contourf(X,Y,u_new_int)
fig.colorbar(cp)
ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()
s=2

In [ ]:
nx=5
ny=5
ip, iu, iv,idu = GenPointer(nx, ny)
np_ = nx * ny
nu = 2*nx*ny - nx - ny
args= uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B,np_,nu,nx,ny,dx,dy,iu,iv,ip,dt,v

#args=np_,nu,nx,ny,dx,dy,iu,iv,ip,dt,v
s=explicit_matrix_combo([Adv],nu,args)


fig,ax=plt.subplots(1,1)
#ax.set_title('$DR^{-1}G$ Combined Operator')
ax.set_title('Advection Operator')
ax.matshow(s)


In [ ]:
s.shape

In [ ]:
s=glob('/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/snapshots/*')
s=[x for x in s if 'ulist' in x]
#s=[int(x.split('ulist_')[-1].split('.')[0]) for x in s ]
#max_u=np.max(s)
u_new_list=[]
v_new_list=[]
#file=f"/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/snapshots/ulist_{max_u}.npy"
for file in s:
    with open(file, 'rb') as f:
        u_new = np.load(f,allow_pickle=True)
        u_new_int,v_new_int=inter_velocity(u_new,nx,ny,iu,iv,idu)
        u_new_int=u_new_int.reshape(nx,ny)
        v_new_int=v_new_int.reshape(nx,ny)
        u_new_list.append(u_new_int)
        v_new_list.append(v_new_int)
print(len(s))

In [ ]:
dat_acq_int=1
HTML(plotter_animation_contour(X,Y,u_new_list,interv=100,dat_acq=dat_acq_int,p=0,q=1).to_html5_video())


In [ ]:
res1=pd.read_csv('/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/res2.csv').rename(columns={'Unnamed: 0':'iteration','0':'Res'})


In [ ]:
#plt.plot(list(res1.iteration),list(res1.Res))

fig,ax=plt.subplots(1,1)
#cp = ax.contour(X, Y, curl_visual,cmap='coolwarm')
plt.plot(list(res1.iteration),list(res1.Res))
ax.set_title('Time Evolution of Residuals')
ax.set_xlabel('Time iterations')
ax.set_ylabel('Residual')
plt.show()
s=2


In [ ]:
v,args,cg_iter,nx,ny,dx,dy,np_,nu,X,Y,ip,iu,iv,idu,dt,p,u,qi,bcL,bcD,uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B=data_unloader(**json.load(open('/Users/moatasimfarooque/Documents/CFD/Moatasim_files/CFD/Project_3/data.json')))
Operator=[[Grad_Vec],[Laplace_Vec],[Div_Vec],[Adv_Vec]]
Operator_names=['Grad','Laplace','Div','Adv']
nu_list=[np_,nu,nu,nu]
#args=np_,nu,nx,ny,dx,dy,iu,iv,ip,dt,v
fig, axs = plt.subplots(2, 2)
k=0
for i in range(2):
    for j in range(2):
        s=explicit_matrix_combo(Operator[k],nu_list[k],args)
        axs[i, j].matshow(s)
        axs[i, j].set_title(f"{Operator_names[k]}")
        k=k+1
plt.show()

In [ ]:
nx=30
ny=29
dx=1/nx
dy=1/ny
ip, iu, iv,idu = GenPointer(nx, ny)
np_ = nx * ny
nu = 2*nx*ny - nx - ny
args=uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B,np_,nu,nx,ny,dx,dy,iu,iv,ip,dt,v

In [ ]:
nx=30
ny=29
ip, iu, iv,idu = GenPointer(nx, ny)
np_ = nx * ny
nu = 2*nx*ny - nx - ny

u_n1=Adv(u_new,*args)

X,Y=np.meshgrid(np.linspace(0,1,nx), np.linspace(0,1,ny), indexing='ij')
u_vec=np.reshape(u_n1[0:iu[-1,-1]+1], (nx-1,ny), order='C')
new_row=np.zeros((ny))
u_vec=np.vstack([u_vec, new_row])

v_vec=np.reshape(u_n1[iu[-1,-1]+1:], (nx,ny-1), order='C')
new_row=np.zeros((nx,1))
v_vec=np.hstack([v_vec, new_row])

fig,ax=plt.subplots(1,1)
#cp = ax.contour(X, Y, curl_visual,cmap='coolwarm')
cp = ax.contourf(X, Y, v_vec)
fig.colorbar(cp)
ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
nx=30
ny=29
ip, iu, iv,idu = GenPointer(nx, ny)
np_ = nx * ny
nu = 2*nx*ny - nx - ny
args=uBC_L, uBC_R, uBC_B, uBC_T, vBC_L, vBC_R, vBC_T, vBC_B,np_,nu,nx,ny,dx,dy,iu,iv,ip,dt,v

u_n=Adv_Vec(u_new,*args)

X,Y=np.meshgrid(np.linspace(0,1,nx), np.linspace(0,1,ny), indexing='ij')
u_vec=np.reshape(u_n[0:iu[-1,-1]+1], (nx-1,ny), order='C')
new_row=np.zeros((ny))
u_vec=np.vstack([u_vec, new_row])

v_vec=np.reshape(u_n[iu[-1,-1]+1:], (nx,ny-1), order='C')
new_row=np.zeros((nx,1))
v_vec=np.hstack([v_vec, new_row])

fig,ax=plt.subplots(1,1)
#cp = ax.contour(X, Y, curl_visual,cmap='coolwarm')
cp = ax.contourf(X, Y, v_vec)
fig.colorbar(cp)
ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (units)')
ax.set_ylabel('y (units)')
plt.show()

In [ ]:
np.allclose(u_n,u_n1)